In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\brett\anaconda3\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [3]:
# Set features. This will also be used as your x values.
X = df[['koi_steff', 'koi_slogg', 'koi_srad']] # 'koi_smet', 'koi_smass', 'koi_sage']]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [4]:
# df['koi_disposition'] = df['koi_disposition'].astype('category')
# df['koi_disposition_cat'] = df['koi_disposition'].cat.codes

# y = df['koi_disposition_cat'].values.reshape(-1, 1)

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()


label_encoder.fit(df['koi_disposition'])
label_encoder.classes_

array(['CANDIDATE', 'CONFIRMED', 'FALSE POSITIVE'], dtype=object)

In [5]:
y_values = label_encoder.transform(df['koi_disposition'])#.astype('int')
y = y_values.reshape(-1, 1)
type(y)

numpy.ndarray

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [7]:
X_train.head()

,koi_steff,koi_slogg,koi_srad
6122,5737,4.327,1.125
6370,5855,4.578,0.797
2879,6328,4.481,0.963
107,4768,4.536,0.779
29,5712,4.359,1.082


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [8]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
X_minmax = MinMaxScaler().fit(X_train)
y_minmax = MinMaxScaler().fit(y_train)

X_train_scaled = X_minmax.transform(X_train)
X_test_scaled = X_minmax.transform(X_test)

print(f"Y Shape: {y_train.shape}")
print(f"X Shape: {X_train_scaled.shape}")

Y Shape: (5243, 1)
X Shape: (5243, 3)


# Train the Model



In [9]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model

SVC(kernel='linear')

In [10]:
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

SVC(decision_function_shape=None, gamma='auto', kernel='linear')

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [12]:
grid.fit(X_train_scaled, y_train.ravel())

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END ..............................C=1, gamma=0.0001; total time=   0.5s
[CV 2/5] END ..............................C=1, gamma=0.0001; total time=   0.6s
[CV 3/5] END ..............................C=1, gamma=0.0001; total time=   0.5s
[CV 4/5] END ..............................C=1, gamma=0.0001; total time=   0.6s
[CV 5/5] END ..............................C=1, gamma=0.0001; total time=   0.6s
[CV 1/5] END ..............................C=1, gamma=0.0005; total time=   0.6s
[CV 2/5] END ..............................C=1, gamma=0.0005; total time=   0.6s
[CV 3/5] END ..............................C=1, gamma=0.0005; total time=   0.6s
[CV 4/5] END ..............................C=1, gamma=0.0005; total time=   0.6s
[CV 5/5] END ..............................C=1, gamma=0.0005; total time=   0.6s
[CV 1/5] END ...............................C=1, gamma=0.001; total time=   0.5s
[CV 2/5] END ...............................C=1,

GridSearchCV(estimator=SVC(kernel='linear'),
             param_grid={'C': [1, 5, 10, 50],
                         'gamma': [0.0001, 0.0005, 0.001, 0.005]},
             verbose=3)

In [13]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 1, 'gamma': 0.0001}
0.5056265872986996


# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash

# import joblib
# filename = 'LogisticRegression.sav'
# joblib.dump(grid, filename)